<a href="https://colab.research.google.com/github/KimJisanER/Dacon/blob/main/metabolism_Discriptor_0830_new_discriptor_sp%2Csp2%2Csp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

[+] 0.0s
conda-forge/linux-64  ⣾  [+] 0

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import Descriptors

In [3]:
def getMolDescriptors(mol, missingVal=None):
    ''' calculate the full list of descriptors for a molecule

        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res


In [4]:
# load data
train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])

test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])

full = pd.concat([train, test], axis = 0).reset_index(drop=True)
full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)

In [5]:
full

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37,<rdkit.Chem.rdchem.Mol object at 0x78d7a536f7d0>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47,<rdkit.Chem.rdchem.Mol object at 0x78d7a536f840>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45,<rdkit.Chem.rdchem.Mol object at 0x78d7a536f8b0>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60,<rdkit.Chem.rdchem.Mol object at 0x78d7a536f920>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43,<rdkit.Chem.rdchem.Mol object at 0x78d7a536f990>
...,...,...,...,...,...,...,...,...,...,...,...,...
3976,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,4.207,306.443,2,1,7,4.207,55.13,<rdkit.Chem.rdchem.Mol object at 0x78d7a54153f0>
3977,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,-0.608,335.398,5,0,1,-1.736,70.16,<rdkit.Chem.rdchem.Mol object at 0x78d7a5415460>
3978,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,1.792,349.383,3,1,3,1.792,69.72,<rdkit.Chem.rdchem.Mol object at 0x78d7a54154d0>
3979,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,0.790,341.132,3,2,2,0.423,69.64,<rdkit.Chem.rdchem.Mol object at 0x78d7a5415540>


In [6]:
from rdkit import DataStructs

allDescrs = [getMolDescriptors(m) for m in full['Molecule']]
full_Descrs = pd.DataFrame(allDescrs)

In [7]:
fragment_counts = {}

def extract_sp3_carbon_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP3:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_sp3_carbon_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')

# 결과 데이터프레임 출력
print(result_df)

       CC  CCO  cCN   Cc  CCN  CC(C)C  CCC  CN(C)C  cS(N)(=O)=O  CN(C)S  ...  \
0     2.0  2.0  1.0  2.0  0.0     0.0  0.0     0.0          0.0     0.0  ...   
1     1.0  0.0  1.0  2.0  1.0     1.0  0.0     0.0          0.0     0.0  ...   
2     1.0  0.0  0.0  0.0  5.0     0.0  1.0     1.0          0.0     0.0  ...   
3     0.0  0.0  0.0  3.0  4.0     0.0  5.0     1.0          1.0     1.0  ...   
4     1.0  0.0  0.0  1.0  1.0     0.0  0.0     0.0          0.0     0.0  ...   
...   ...  ...  ...  ...  ...     ...  ...     ...          ...     ...  ...   
3976  3.0  0.0  0.0  0.0  1.0     0.0  6.0     0.0          0.0     0.0  ...   
3977  2.0  0.0  0.0  0.0  6.0     0.0  2.0     1.0          0.0     0.0  ...   
3978  2.0  0.0  0.0  0.0  1.0     0.0  0.0     0.0          0.0     0.0  ...   
3979  0.0  0.0  0.0  0.0  0.0     0.0  0.0     0.0          0.0     0.0  ...   
3980  2.0  0.0  0.0  0.0  5.0     1.0  1.0     2.0          0.0     0.0  ...   

      cC(C)(c)C  cC(C)(N)O  c[C@@H](C)n

In [8]:
fragment_counts = {}

def extract_non_ring_sp2_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP2 and atom.IsInRing() == False:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_non_ring_sp2_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df2 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df2.columns = ['non_ring_sp2_' + column for column in result_df2.columns]

# 결과 데이터프레임 출력
print(result_df2)


      non_ring_sp2_COc  non_ring_sp2_CNC  non_ring_sp2_cC(N)=O  \
0                  1.0               1.0                   1.0   
1                  0.0               0.0                   0.0   
2                  0.0               0.0                   0.0   
3                  0.0               0.0                   0.0   
4                  0.0               0.0                   1.0   
...                ...               ...                   ...   
3976               0.0               1.0                   0.0   
3977               0.0               0.0                   0.0   
3978               0.0               1.0                   0.0   
3979               0.0               0.0                   1.0   
3980               0.0               0.0                   0.0   

      non_ring_sp2_C=O  non_ring_sp2_cOC  non_ring_sp2_O=S  non_ring_sp2_c=O  \
0                  1.0               1.0               0.0               0.0   
1                  1.0               0.0       

In [9]:
fragment_counts = {}

def extract_sp_carbon_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_sp_carbon_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df3 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df3.columns = ['sp_' + column for column in result_df3.columns]

# 결과 데이터프레임 출력
print(result_df3)

      sp_cC#N  sp_C#N  sp_CC#N  sp_C#CN  sp_C#Cc  sp_C#Cn  sp_N#CN  sp_C#CC
0         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
1         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
2         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
3         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
4         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
...       ...     ...      ...      ...      ...      ...      ...      ...
3976      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
3977      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
3978      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
3979      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
3980      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0

[3981 rows x 8 columns]


In [10]:
fragment_counts = {}

def extract_ring_sp2_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP2 and atom.IsInRing() == True:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_ring_sp2_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df4 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df4.columns = ['ring_sp2_' + column for column in result_df4.columns]

# 결과 데이터프레임 출력
print(result_df4)


      ring_sp2_cc(c)O  ring_sp2_ccc  ring_sp2_cc(c)C  ring_sp2_cc(C)[nH]  \
0                 2.0           4.0              1.0                 1.0   
1                 0.0           4.0              0.0                 0.0   
2                 0.0           4.0              0.0                 0.0   
3                 0.0           6.0              1.0                 0.0   
4                 1.0           5.0              2.0                 0.0   
...               ...           ...              ...                 ...   
3976              0.0           0.0              1.0                 0.0   
3977              0.0           0.0              0.0                 0.0   
3978              0.0           8.0              3.0                 0.0   
3979              1.0           3.0              1.0                 0.0   
3980              0.0           2.0              0.0                 0.0   

      ring_sp2_cc(-c)n  ring_sp2_cc(-c)s  ring_sp2_csc  ring_sp2_Cc(n)s  \
0           

In [14]:
Data = pd.concat([full, full_Descrs, result_df, result_df2, result_df3, result_df4], axis=1).drop(columns=['AlogP', 'Molecule', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])


missing_indices = train[train.isnull().any(axis=1)].index

train = Data.iloc[:3498].dropna(axis=0).dropna(axis=1)
test = Data.iloc[3498:].drop(columns=missing_indices)

In [21]:
corr = Data.iloc[:, 2:].corr().sort_values(by='MLM')
corr = corr[['MLM', 'HLM']].dropna(axis=0)

In [27]:
corr.head(40)

,MLM,HLM
LogD,-0.350146,-0.357456
VSA_EState6,-0.191638,-0.200428
PEOE_VSA6,-0.180522,-0.193464
PEOE_VSA7,-0.171349,-0.195136
ring_sp2_ccc,-0.170961,-0.189449
EState_VSA7,-0.164922,-0.148497
NumAromaticCarbocycles,-0.153674,-0.177827
fr_benzene,-0.153674,-0.177827
SMR_VSA7,-0.152810,-0.189298
SlogP_VSA6,-0.152545,-0.174692


In [28]:
corr.tail(40)

,MLM,HLM
fr_amide,0.080684,0.041589
fr_Ar_N,0.083670,0.099096
ring_sp2_cnn,0.084892,0.087549
qed,0.086126,0.175906
fr_C_O,0.092443,0.007220
non_ring_sp2_C=O,0.092443,0.007220
EState_VSA1,0.096722,0.016369
NumSaturatedHeterocycles,0.098281,0.090644
ring_sp2_c[nH]c,0.103039,0.070084
PEOE_VSA13,0.106473,0.076461


In [40]:
train

,id,SMILES,MLM,HLM,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,ring_sp2_CN(O)S,ring_sp2_Cc(c)n,ring_sp2_CC(n)=O,ring_sp2_nc(n)Br,ring_sp2_cc([N+])n,ring_sp2_C=c(c)n,ring_sp2_nno,ring_sp2_cc(F)s,ring_sp2_cc(o)O,ring_sp2_C=C(C)Br
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,0.192138,0.333333,0.2,0.210526,0.579660,0.389270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,0.108751,0.133333,0.1,0.052632,0.502622,0.239538,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,0.105343,0.333333,0.0,0.078947,0.461021,0.201951,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,0.271376,0.400000,0.0,0.131579,0.594968,0.304785,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,0.080898,0.200000,0.0,0.026316,0.514316,0.133668,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,0.188519,0.200000,0.1,0.131579,0.590291,0.209762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,0.157539,0.266667,0.1,0.078947,0.479376,0.252839,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,0.075016,0.200000,0.1,0.157895,0.499220,0.228180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.094688,0.333333,0.1,0.131579,0.418781,0.301068,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
duplicates = full[full.duplicated(subset='SMILES', keep=False)].sort_values(by='SMILES')

In [44]:
duplicates

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
2276,TRAIN_2276,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,1.535,31.453,3.556,262.309,3,0,4,3.556,43.60,<rdkit.Chem.rdchem.Mol object at 0x7ab6f4a59690>
451,TRAIN_0451,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.310,24.670,3.556,262.309,3,0,4,3.556,43.60,<rdkit.Chem.rdchem.Mol object at 0x7ab6f4c230d0>
2891,TRAIN_2891,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,55.950,69.950,2.172,337.372,4,2,3,2.169,82.00,<rdkit.Chem.rdchem.Mol object at 0x7ab6f4a6a6c0>
543,TRAIN_0543,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,68.485,85.872,2.172,337.372,4,2,3,2.169,82.00,<rdkit.Chem.rdchem.Mol object at 0x7ab6f4c259a0>
837,TRAIN_0837,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,63.522,62.488,2.293,367.428,5,2,3,2.307,139.85,<rdkit.Chem.rdchem.Mol object at 0x7ab6f4a31b60>
366,TRAIN_0366,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,73.740,66.850,2.293,367.428,5,2,3,2.307,139.85,<rdkit.Chem.rdchem.Mol object at 0x7ab6f4c20ba0>
1085,TRAIN_1085,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,23.630,71.470,1.684,381.450,7,1,4,1.684,126.52,<rdkit.Chem.rdchem.Mol object at 0x7ab6f4a38900>
2848,TRAIN_2848,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,40.657,99.900,1.684,381.450,7,1,4,1.684,126.52,<rdkit.Chem.rdchem.Mol object at 0x7ab6f4a693f0>
2096,TRAIN_2096,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,1.880,61.280,2.843,360.490,3,0,4,2.843,49.85,<rdkit.Chem.rdchem.Mol object at 0x7ab6f4a54740>
1666,TRAIN_1666,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,5.494,25.397,2.843,360.490,3,0,4,2.843,49.85,<rdkit.Chem.rdchem.Mol object at 0x7ab6f4a48970>


In [45]:
print(train['SMILES'][2797])

COc1cc(CN2CCC(NC(=O)c3cc4cc(Br)c5ccccc5c4o3)CC2)cc(OC)c1OC


In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3496 entries, 0 to 3497
Columns: 780 entries, id to ring_sp2_C=C(C)Br
dtypes: float64(674), int64(104), object(2)
memory usage: 20.8+ MB


In [31]:
#결측 확인
for col in train.columns:
    has_missing_values = train[col].isna().any()
    count =  train[col].isna().sum()
    if has_missing_values:
        print(f"Column '{col}' has missing values.")
        print(f"'{count}'")

In [32]:
for col in test.columns:
    has_missing_values = test[col].isna().any()
    count =  test[col].isna().sum()
    if has_missing_values:
        print(f"Column '{col}' has missing values.")
        print(f"'{count}'")

Column 'MLM' has missing values.
'483'
Column 'HLM' has missing values.
'483'


In [33]:
# 숫자형식 컬럼들의 min-max 정규화
scaler = MinMaxScaler()
non_features = ['id', 'SMILES', 'MLM', 'HLM','Fingerprint']
features = [column for column in train.columns if column not in non_features]
train[features] = scaler.fit_transform(train[features])

test[features] = scaler.transform(test[features])

In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import math
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define features and targets
non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint']
features = [column for column in train.columns if column not in non_features]
mlm_target = "MLM"
hlm_target = "HLM"

# Initialize KFold
seed = 42
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

# Initialize arrays to store models and scores
reg_mlms = []
reg_hlms = []

# Initialize arrays to store RMSE scores
mlm_rmse_scores = []
hlm_rmse_scores = []

# Loop through KFold splits
for i, (train_index, valid_index) in enumerate(kf.split(train)):
    df_train = train.iloc[train_index]
    df_valid = train.iloc[valid_index]

    x_train_num = df_train[features].values

    y_mlm_train = df_train[mlm_target].values
    y_hlm_train = df_train[hlm_target].values

    x_valid_num = df_valid[features].values

    y_mlm_valid = df_valid[mlm_target].values
    y_hlm_valid = df_valid[hlm_target].values

    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(256, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    merged = layers.Dense(1024, activation='relu')(x1)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model = keras.Model(inputs=x1_input, outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

    # Create and compile another model for HLM
    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(256, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    merged = layers.Dense(1024, activation='relu')(x1)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model_hlm = keras.Model(inputs=x1_input, outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model_hlm.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])


    # Train the model
    checkpoint_mlm = ModelCheckpoint(f'model_mlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_mlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model.fit(x_train_num, y_mlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_mlm_valid),
              callbacks=[checkpoint_mlm, early_stopping_mlm])

    reg_mlms.append(model)

    checkpoint_hlm = ModelCheckpoint(f'model_hlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_hlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model_hlm.fit(x_train_num, y_hlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_hlm_valid),
                  callbacks=[checkpoint_hlm, early_stopping_hlm])

    reg_hlms.append(model_hlm)

    # Calculate RMSE for MLM predictions
    y_mlm_pred = model.predict(x_valid_num)
    mlm_rmse = math.sqrt(mean_squared_error(y_mlm_valid, y_mlm_pred))
    print(mlm_rmse)
    mlm_rmse_scores.append(mlm_rmse)

    # Calculate RMSE for HLM predictions
    y_hlm_pred = model_hlm.predict(x_valid_num)
    hlm_rmse = math.sqrt(mean_squared_error(y_hlm_valid, y_hlm_pred))
    print(hlm_rmse)
    hlm_rmse_scores.append(hlm_rmse)

Epoch 1/500
197/197 [==============================] - 3s 7ms/step - loss: 1294.3074 - root_mean_squared_error: 35.9765 - val_loss: 1010.7677 - val_root_mean_squared_error: 31.7926
Epoch 2/500
197/197 [==============================] - 1s 6ms/step - loss: 995.0460 - root_mean_squared_error: 31.5443 - val_loss: 894.2139 - val_root_mean_squared_error: 29.9034
Epoch 3/500
197/197 [==============================] - 1s 6ms/step - loss: 918.1516 - root_mean_squared_error: 30.3010 - val_loss: 842.3055 - val_root_mean_squared_error: 29.0225
Epoch 4/500
197/197 [==============================] - 1s 6ms/step - loss: 877.9600 - root_mean_squared_error: 29.6304 - val_loss: 853.6514 - val_root_mean_squared_error: 29.2173
Epoch 5/500
197/197 [==============================] - 1s 6ms/step - loss: 838.0185 - root_mean_squared_error: 28.9485 - val_loss: 857.1812 - val_root_mean_squared_error: 29.2777
Epoch 6/500
197/197 [==============================] - 1s 6ms/step - loss: 822.2685 - root_mean_squared

In [35]:
from keras.models import load_model

# Load MLM models
mlm_models = [load_model(f'model_mlm_fold{i}.h5') for i in range(n_splits)]

# Load HLM models
hlm_models = [load_model(f'model_hlm_fold{i}.h5') for i in range(n_splits)]

x_test = test[features].values

mlm_predictions = []
hlm_predictions = []

for model_mlm, model_hlm in zip(mlm_models, hlm_models):
    mlm_predictions.append(model_mlm.predict(x_test))
    hlm_predictions.append(model_hlm.predict(x_test))

# Convert prediction lists to numpy arrays
mlm_predictions = np.array(mlm_predictions)
hlm_predictions = np.array(hlm_predictions)

mlm_ensemble_prediction = mlm_predictions.mean(axis=0)
hlm_ensemble_prediction = hlm_predictions.mean(axis=0)

16/16 [==============================] - 0s 2ms/step


In [36]:
df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")
df_submission["MLM"] = mlm_ensemble_prediction
df_submission["HLM"] = hlm_ensemble_prediction
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")

In [37]:
df_submission

,id,MLM,HLM
0,TEST_000,15.765401,35.811340
1,TEST_001,69.861786,85.355690
2,TEST_002,39.061626,49.095634
3,TEST_003,60.840900,76.907578
4,TEST_004,52.535866,75.789200
...,...,...,...
478,TEST_478,7.983827,24.167704
479,TEST_479,77.098099,92.199638
480,TEST_480,43.392902,64.302734
481,TEST_481,56.453072,67.513474
